# Bienvenue sur le meilleur notebook

In [149]:
import numpy as np
from matplotlib import pyplot as plt
import math

# Calculs de bornes primales

In [39]:
def computeCost(p,d,w,order):
    t=0
    cost=0
    for idx in order:
        t += p[idx]
        cost += max(0,(t-d[idx])*w[idx])
    return cost

In [86]:
def getPrimal_0(p,d,w):
    sortedTasks=np.argsort(d)
    cost=computeCost(p,d,w,sortedTasks)
    return(cost,list(sortedTasks))

In [87]:
def getPrimal_1(p,d,w):
    sortedTasks=(np.argsort((np.array(d)-np.array(p))*np.array(w)))[::-1]
    cost=computeCost(p,d,w,sortedTasks)
    return(cost,list(sortedTasks))

In [88]:
def getPrimal_2(p,d,w):
    tasksLeft=[int(i) for i in range(len(p))]
    order=[]
    while len(tasksLeft)!= 0:
        time=np.sum(np.array(p)[tasksLeft])
        penalities=(time-np.array(d)[tasksLeft])*np.array(w)[tasksLeft]
        sortedTasks=np.argsort(penalities)
        order.append(tasksLeft[sortedTasks[0]])
        tasksLeft.pop(sortedTasks[0])
    order.reverse()
    cost = computeCost(p,d,w,order)
    return(cost,order)

## Test des différentes méthodes : 

In [91]:
w=[4,5,3,5]
d=[16,26,25,27]
p=[12,8,15,9]
print("Classement par ordre croissant des dates d'usinage au plus tard :\n",getPrimal_0(p,d,w))
print("Classement par ordre décroissant des (d-p)*w : \n",getPrimal_1(p,d,w))
print("Classement en mettant en dernière position disponible l'élémen non traité donnant lieu à la plus petite pénalité de retard : \n",getPrimal_2(p,d,w))

Classement par ordre croissant des dates d'usinage au plus tard :
 (136, [0, 2, 1, 3])
Classement par ordre décroissant des (d-p)*w : 
 (133, [3, 1, 2, 0])
Classement en mettant en dernière position disponible l'élémen non traité donnant lieu à la plus petite pénalité de retard : 
 (67, [0, 1, 3, 2])


# Calculs de borne dual

In [138]:
def getDual_0(p,d,w,visited):
    dual=0
    time=0
    notVisited=list(range(len(p)))
    for i in visited:
        retard=((p[i]+time)-d[i])
        dual+=retard*w[i]*(retard>0)
        time+=p[i]
        notVisited.remove(i)
        
    return dual

In [139]:
def getDual_1(p,d,w,visited):
    dual=0
    time=0
    notVisited=list(range(len(p)))
    for i in visited:
        retard=((p[i]+time)-d[i])
        dual+=retard*w[i]*(retard>0)
        time+=p[i]
        notVisited.remove(i)
        
    for j in notVisited:
        retard=(p[j]+time)-d[j]
        dual+=retard*w[j]*(retard>0)
    return dual

In [140]:
def getDual_2(p,d,w,visited):
    dual=0
    time=np.sum(p)

    for i in visited:
        retard=(time-d[i])
        dual+=retard*w[i]*(retard>0)
        time-=p[i]
        
    return dual

## Test des méthodes : 

In [141]:
print("Si on a fixé l'ordre de k pièces on calcule les pénalités de retard de ces k pièces : \n",getDual_0(p,d,w,[0,1]))
print("Si on a fixé l'ordre de k pièces on calcule les pénalités de retard de ces k pièces et on ajoute les pénalités des autres pièces en supposant qu'elles sont en (k+1)ème position : \n",getDual_1(p,d,w,[0,1]))

Si on a fixé l'ordre de k pièces on calcule les pénalités de retard de ces k pièces : 
 0
Si on a fixé l'ordre de k pièces on calcule les pénalités de retard de ces k pièces et on ajoute les pénalités des autres pièces en supposant qu'elles sont en (k+1)ème position : 
 40


# Implementation 

In [146]:
class Node:
    def __init__(self, parent, children, lb, visited):
        self.parent = parent
        self.children = children
        self.lb = lb
        self.visited = visited
    def __str__(self):
        return "Parent : " +str(self.parent)+"\n Children : "+str(self.children) + "\n lb : "+str(self.lb) + "\n visited : "+str(self.visited)

In [266]:
def branch_and_bound(p,d,w,getPrimal,getDual,bylast,steps):
    
    nbIterations=0
    Tasks = list(range(len(p)))
    Tree = [] # list of all nodes created (a list of Node objects)
    Queue = [] # list of nodes to process (a list of integers with the index of nodes to process in the Tree)
    UB = 10000000 # set the upper bound to a sufficiently large number
    LB = 0 # set the lower bound to a sufficiently small number 
    
    UB,currentOrder = getPrimal(p,d,w)
    if(steps):
        print("First UB : ", UB)
    
    root = Node(None, [], 0, []) # at the root node no variables are fixed 
    Tree.append(root) # start the tree with the root node
    Queue.append(0) # start the list of nodes to process with the root node (note that index of root in Tree is 0)
    
    #continue processing nodes until the list is empty OR 
    #until an optimality tolerance between the lower bound and the upper bound is reached
    
    while Queue!=[]:
        nbIterations+=1
        # process the first node in the queue
        currentIndex = Queue[0]
        currentNode = Tree[currentIndex]
        
        if(steps):
            print("\n##################################")
            print("Current Node : {}\n".format(currentIndex),currentNode )
            print("----------------------------------")
        
        if len(currentNode.visited) == len(p):
            branching=2
            order = currentNode.visited.copy()
            if(bylast==True):
                order.reverse()
            solutionCost = computeCost(p,d,w,order)
            if solutionCost <= UB:
                UB = solutionCost
                currentOrder=order
                
        if currentNode.lb < UB:
            
            branching=1
            
            for nextTask in (set(Tasks)-set(currentNode.visited)):
                nextVisited = currentNode.visited.copy()
                nextVisited.append(nextTask)
                nextLb = getDual(p,d,w, nextVisited)
                
                newNode = Node(currentIndex, [], nextLb,nextVisited)
                Tree.append(newNode)
                Queue.append(len(Tree)-1)
                currentNode.children.append(len(Tree)-1)
                
        else :
            branching=0
        
        # Calculate the current LB as the maximum of all the upper bounds among the active nodes
        currentLB = -math.inf
        for node in Tree:
            if node.lb > currentLB and node.lb <= UB:
                currentLB = node.lb
        
        # Update the LB if the current LB is better
        if currentLB >= LB:
            LB = currentLB
        
        if(steps):
            # Display the current upper and lower bounds    
            print("LB= ",LB," UB= ", UB)
            # When the processing of the node is completed remove the node from the queue     
        Queue.remove(currentIndex)
        
        if(steps):
            if branching==0:
                print("##################################")
                print("\nPRUNE")
        
            elif branching==1 and LB!=UB:
                print("##################################")
                print("\nBRANCHING")
            
        


    # return the  solution 
    if(steps):
        print("##################################")
        print("\nEND")
    return UB,currentOrder,nbIterations

In [267]:
branch_and_bound(p,d,w,getPrimal_1,getDual_1,bylast=False,steps=False)

(67, [0, 1, 3, 2], 48)

In [268]:
branch_and_bound(p,d,w,getPrimal_2,getDual_2,bylast=True,steps=False)

(67, [0, 1, 3, 2], 8)

In [269]:
branch_and_bound(p,d,w,getPrimal_0,getDual_1,bylast=False,steps=False)

(67, [0, 1, 3, 2], 52)

In [270]:
branch_and_bound(p,d,w,getPrimal_0,getDual_0,bylast=False,steps=False)

(67, [0, 1, 3, 2], 65)